In [2]:
from flask import Flask, request, jsonify
import os
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

app = Flask(__name__)

# Set environment variables
os.environ['OPENAI_API_KEY'] = ''
os.environ['PINECONE_API_KEY'] = 'f5fa0fda-521f-4e2b-959f-0ba03ea28de3'

# Initialize OpenAI LLM
llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])

# Pinecone index name
index_name = "hackbangalore-rag-trial-01"

# Initialize Pinecone vector store
vectorstore = PineconeVectorStore(index_name=index_name,
                                  embedding=OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY']))


def extract_keywords_using_chatgpt(text):
    """
    Sends a request to ChatGPT to extract keywords from the provided text.

    :param text: The text from which keywords need to be extracted.
    """
    try:
        # Invoke the LLM to get the response, which is expected to be directly the keyword string
        response = llm.invoke(
            "Extract the main keywords from the following text and provide them as a comma-separated list. do not give more than 10 keywords Please prefix the keywords with 'Keywords:'. the text to extract keyword is, " + text
        )
        print("Response from ChatGPT:", response)  # Debug: Print the full response to verify its structure
        print("Type of response:", type(response))
        print("Content of response:", response)

        # Check if the response is directly usable
        # strip /n and /r from the response
        response = response.replace('\n', '').replace('\r', '')
        if isinstance(response, str) and response.startswith('Keywords:'):
            print("Response is directly usable")
            keywords = response[len('Keywords:'):].strip()
            return [keyword.strip() for keyword in keywords.split(',')]
        elif isinstance(response, dict) and 'text' in response:
            output = response['text'].strip()
            if output.startswith('Keywords:'):
                keywords = output[len('Keywords:'):].strip()
                return [keyword.strip() for keyword in keywords.split(',')]
        else:
            print("Unexpected response structure or no keywords prefix found")
            return []
    except Exception as e:
        print(f"Error processing response: {e}")
        return []


@app.route('/search_projects', methods=['POST'])
def search_projects():
    try:
        data = request.get_json()
        description = data.get('description', '')

        # Extract keywords relevant to the description
        keywords = extract_keywords_using_chatgpt(description)

        # Perform a similarity search using the vector store with extracted keywords
        search_query = ' '.join(keywords)
        print(f"Search query: {search_query}")
        search_result = vectorstore.similarity_search(search_query)

        # Return the top 3 results, if more than 3 exist
        top_results = search_result[:3] if len(search_result) > 3 else search_result

        if top_results:
            response_data = [{"similarity_match": result.page_content} for result in top_results]
            return jsonify(response_data), 200
        else:
            return jsonify({'message': 'No results found'}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500


if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/Zaid/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/Zaid/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 18, in <module>
    from IPython.core.application import (
  File "/Users/Zaid/opt/anaconda3/lib/python3.9/site-packages/IPython/__init__.py", line 56, in <module>
    from .terminal.embed import embed
  File "/Users/Zaid/opt/anaconda3/lib/python3.9/site-packages/IPython/terminal/embed.py", line 16, in <module>
    from IPython.terminal.interactiveshell import TerminalInteractiveShell
  File "/Users/Zaid/opt/anaconda3/lib/python3.9/site-packages/IPython/terminal/interactiveshell.py", line 35, in <module>
    from .debugger import TerminalPdb, Pdb
  File "/Users/Zaid/opt/anaconda3/lib/python3.9/site-packages/IPython/t

SystemExit: 1

/Users/Zaid/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
